<a href="https://colab.research.google.com/github/zhiyuan-95/House-Prices/blob/main/neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install keras-tuner -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 3.1 MB/s eta 0:00:00


In [2]:
from tensorflow import keras
import tensorflow as tf
from keras_tuner.tuners import RandomSearch
import pandas as pd
from sklearn.model_selection import train_test_split
from keras import layers
import os
import random

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:
feature = pd.read_csv(f'/content/drive/MyDrive/Colab/kaggle/HousePrices/feature_train_stacked.csv')
target = pd.read_csv(f'/content/drive/MyDrive/Colab/kaggle/HousePrices/target_tran.csv')
test = pd.read_csv(f'/content/drive/MyDrive/Colab/kaggle/HousePrices/test_stacked.csv')

In [5]:
from keras import backend as K

In [6]:
class nn_tuner:
    def __init__(self, n_layers):
      self._n_layers = n_layers
      self._record_path = f'/content/drive/MyDrive/Colab/kaggle/HousePrices/nn record/{self._n_layers+2}_layers.csv'
      column_d = ['dense_{0}'.format(n) for n in range(self._n_layers)]
      column_a = ['activation_{0}'.format(n) for n in range(self._n_layers)]
      self._columns = column_d+column_a+['epochs']+['batch_size']+['ln']+['score']
    def reset(self):
      df = pd.DataFrame(columns = self._columns)
      df.to_csv(self._record_path, index = False)
    def update(self,params,score):
      record = pd.read_csv(self._record_path)
      record.loc[len(record)] = params
      record.loc[len(record)-1,'score'] = score
      record.to_csv(self._record_path, index=False)
    def top3(self):
      records = pd.read_csv(self._record_path)
      return records.sort_values(by = 'score').iloc[:3,:]
    def get_record(self):
      records = pd.read_csv(self._record_path)
      return records.sort_values(by = 'score')
    def best_param(self):
      records = pd.read_csv(self._record_path)
      return records.sort_values(by = 'score').iloc[0,:-1]

In [7]:
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))
def model(X_train,y_train,X_val,y_val,ds,act_fn,ln,ep,bs):
    md = tf.keras.models.Sequential()
    md.add(layers.Flatten(input_shape=(317,))),
    md.add(layers.BatchNormalization()),
    md.add(layers.Dense(ds[0], activation=act_fn[0]))
    md.add(layers.BatchNormalization()),
    md.add(layers.Dense(ds[1], activation=act_fn[1]))
    md.add(layers.Dense(1))
    md.compile(loss="mean_absolute_error",
              optimizer=tf.keras.optimizers.Adam(learning_rate=ln),
              metrics=[rmse])
    md.fit(X_train,y_train,
           epochs = ep,
           batch_size=bs,
           shuffle=True,
           validation_data = (X_val,y_val), verbose=0)
    return sum(md.history.history['val_rmse'][-3:])/3

In [ ]:
# didn't count the input and output layers
my_tuner = nn_tuner(2)
#my_tuner.reset()
number_of_trials = 100
executions_per_trial = 3
for _ in range(number_of_trials):
  ds = [random.randint(200,700) for d in range(2)]
  act_fn = [{0:'elu',1:'sigmoid',2:'tanh',3:'softmax'}[random.randint(0,3)],'elu']
  ln = round(random.uniform(0.01,0.1),3)
  ep = random.randint(4,40)*5
  bs = random.randint(6,40)*5
  params = {'ds':ds,'act_fn':act_fn,'ln':ln,'ep':ep,'bs':bs}
  score = []
  print('#_{0}_'.format(_), end=' ')
  for n in ds+act_fn+[ep]+[bs]+[ln]:print(n, end=' ')
  for __ in range(executions_per_trial):
    X_train, X_val, y_train, y_val = train_test_split(feature, target, test_size=0.2, random_state=42)
    score.append(model(X_train,y_train,X_val,y_val, **params))
    print('-> ',end='')
  print(round(sum(score)/3,2))
  my_tuner.update({i:j for i,j in zip(my_tuner._columns,ds+act_fn+[ep]+[bs]+[ln])},round(sum(score)/3,2))
my_tuner.top3()

#_0_ 208 351 softmax elu 85 60 0.092 -> -> -> 61625.6
#_1_ 586 566 tanh elu 180 115 0.02 -> -> -> 43142.33
#_2_ 533 675 sigmoid elu 175 200 0.08 -> -> -> 29879.85
#_3_ 488 235 softmax elu 180 45 0.079 -> -> -> 45608.98
#_4_ 317 390 tanh elu 65 30 0.055 -> -> -> 76994.71
#_5_ 358 325 tanh elu 20 55 0.053 -> -> -> 42424.47
#_6_ 582 597 sigmoid elu 105 150 0.035 -> -> -> 32140.18
#_7_ 547 471 sigmoid elu 85 45 0.081 -> -> -> 38789.75
#_8_ 632 310 softmax elu 150 155 0.058 -> -> -> 40360.38
#_9_ 507 345 elu elu 60 55 0.043 -> -> -> 28597.01
#_10_ 655 240 softmax elu 130 85 0.062 -> -> -> 40012.04
#_11_ 602 615 tanh elu 170 50 0.014 -> -> -> 55693.76
#_12_ 212 301 tanh elu 120 195 0.086 -> -> -> 29747.08
#_13_ 238 630 softmax elu 155 95 0.067 -> -> -> 42366.57
#_14_ 204 560 elu elu 170 35 0.036 -> -> -> 56276.59
#_15_ 212 309 tanh elu 70 145 0.095 -> -> -> 44926.09
#_16_ 699 633 tanh elu 20 155 0.062 -> -> -> 109169.07
#_17_ 410 401 softmax elu 70 150 0.027 -> -> -> 37686.73
#_18_ 383 459 e

In [9]:
my_tuner.top10()

,dense_0,dense_1,activation_0,activation_1,epochs,batch_size,ln,score
31,388,551,elu,elu,160,140,0.0300,25085.62
21,537,286,elu,elu,105,155,0.0741,25465.52
27,271,344,elu,elu,200,180,0.0261,25799.79
44,255,387,softmax,elu,195,135,0.0110,26207.92
28,336,633,elu,elu,180,165,0.0467,26208.41
73,604,359,tanh,elu,135,165,0.0540,26953.04
40,298,569,elu,elu,155,185,0.0173,27036.71
36,248,289,elu,elu,95,170,0.0238,27084.32
67,357,390,sigmoid,elu,95,170,0.0850,27238.39
10,649,455,elu,elu,65,55,0.0153,27459.45


In [ ]:
md = tf.keras.models.Sequential()
md.add(layers.Flatten(input_shape=(317,))),
md.add(layers.BatchNormalization()),
md.add(layers.Dense(537, activation='elu'))
md.add(layers.BatchNormalization()),
md.add(layers.Dense(286, activation='elu'))
md.add(layers.Dense(1))
md.compile(loss="mean_absolute_error",
          optimizer=tf.keras.optimizers.Adam(
              learning_rate=0.04),
          metrics=[rmse])
md.fit(X_train,y_train,
        epochs = 105,
        batch_size=155,
        shuffle=True,validation_data = (X_val,y_val),verbose=1)


In [15]:
md.history.history['val_rmse'][-1]

24187.46484375

In [22]:
prediction = md.predict(test).T[0]
result =pd.DataFrame({'Id':[int(1461+x) for x in range(len(prediction))], 'SalePrice':prediction})
result.to_csv('submission.csv',index=False )

46/46 [==============================] - 0s 2ms/step


In [24]:
from google.colab import files
# Upload the kaggle.json file
files.upload()
!mkdir /root/.kaggle/
!cp kaggle.json /root/.kaggle/

Saving kaggle.json to kaggle.json
mkdir: cannot create directory ‘/root/.kaggle/’: File exists


In [25]:
!kaggle competitions submit -c house-prices-advanced-regression-techniques -f submission.csv -m "Message"

100% 21.1k/21.1k [00:00<00:00, 35.9kB/s]
Successfully submitted to House Prices - Advanced Regression Techniques